In [ ]:
%run preprocecssing_helpers.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})
# matplotlib.rcParams.

In [ ]:
# path = 'F:\StroopEEG'
path = 'F:\EEGLAB'

# eeglab_files = os.listdir(path)
# eeglab_dict = search_folders(path, eeglab_files)
# eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
# eeglab_files

In [ ]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [ ]:
# raw = mne.io.read_raw_eeglab(eeglab_files[2],preload = True)

In [ ]:
def list_eeglab_files(root_path):
    """
    NOTE: must perform magic command %run preprocecssing_helpers.ipynb before using this method.
    
    Description: 
        Lists all eeglab .set files within a directory. 
        Assuming that the file format is: 
        Root Folder -> Subject Folder -> eeglab file contents
        example: 
        EEGLAB (root) -> SA20140220 (subject) -> 20140220_1009.set
        or 
        F:\\EEGLAB\\SA20140220\\20140220_1009.set
    
    Variables: 
        root_path: the folder containing all subject folders
    ----
    Returns:
        List of file .set path locations.
    """
    
    eeglab_files_list = os.listdir(root_path)
    eeglab_dict = search_folders(root_path, eeglab_files_list)
    eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
    
    return eeglab_files

In [ ]:
# eeglab_files = list_eeglab_files(path)

In [ ]:
def find_events(eeglab):
    """
    Description:
        returns the events of an eeglab .set object
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns: np array of 3d lists
    """
    return mne.find_events(eeglab, output= 'offset',
#                               consecutive= True,
                              min_duration=1/eeglab.info['sfreq'], 
                              shortest_event= .5, 
                              initial_event=False)
    

In [ ]:
# events = find_events(raw)
# type(events)

In [ ]:
def get_trials(events):
    """
    Descriptions:
        returns a list of trials where each trial contains timestamp, offset and event_id
    
    Variables:
        events: A np array of 3d lists. use mne.find_events or custom find_events(eeglab) method already written.
    -----
    returns: np array of lists
    """
    trials = []
    current_trial = []
    for event in events:
        if event[-1] == 31 and current_trial != []:
            trials.append(current_trial)
            current_trial = []
            current_trial.append(event)
        else:
            current_trial.append(event)
            
    trials.append(current_trial) # the last trial does not have an indicator, so append it after the loop ends
    return trials

In [ ]:
# trials = get_trials(events)

In [ ]:
# trials[1][0]

In [ ]:
def filter_bad_trials(eeglab):
    """
    Description:
        Filters out all events that are not within the 1-24 stim-code definitions.
        Also filters for trials that have a reaction time greater than .500ms as the subject 
        failed to click in time and was marked incorrect anyway.
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns:
        new_events: 
            np array of 3d list of events 
    """
    
    events = find_events(eeglab)
    trials = get_trials(events)
    button_clicks = [131,132]
    accuracy = [129,130]
    found_first_stim_code = False
    found_third_stim_code = False
    isEnd_of_trial = False
    new_events = []
    good_trials = []
    bad_trials_count = 0
    for trial in trials:
#         print(trial)
        for event in trial:
            # find first stim-code
            if event[-1] <= 24 and found_first_stim_code == False: 
                first_stim_code = event
                found_first_stim_code = True
#                 print("found first stim-code: {0}".format(first_stim_code))

            # find third stim-code
            if found_first_stim_code and event[-1] <= 24:
                #the third stim-code is one less than the first. 
                if event[-1] + 1 == first_stim_code[-1]:
                    third_stim_code = event
                    found_third_stim_code = True
            
            if event[-1] in button_clicks:
                button_click = event        
                
            if event[-1] in accuracy:
                #end of trial
                if event[-1] == 130: # incorrect
                    button_click = event
                    time_diff = (button_click[0] - third_stim_code[0]) * (1/512) #compute time in seconds
                    if time_diff > 1:
                        print("time difference {0}".format(time_diff))
                        print(third_stim_code)
                        print(button_click)
                        bad_trials_count+=1
                    else:
                        #store this trial
                         good_trials.append(trial)
                else:
                    good_trials.append(trial)
                found_first_stim_code = False
                found_third_stim_code = False
                
    print("\n **{0} bad trials dropped**".format(bad_trials_count))
    return np.concatenate([x for x in good_trials])

In [ ]:
# filtered_events = filter_bad_trials(raw)

In [ ]:
def events_with_stim_codes(events):
    """
    Description:
        Method will return an mne events ndarray containing only 
        stim-codes (event_ids 1-24).
    Variables:
        events: list of 3d lists 
    -----
    Returns:
        new_events list of 3d lists containing events of stim-codes    
    """
    new_events = []
    for event in events:
        if event[-1] <= 24:
            new_events.append(np.array(event))
            
    return np.array(new_events)
    

In [ ]:
# some eeglab .set files may not have seen ALL of the 1-24 stim-code events.
# write method to return a proper event_id dictionary
def get_stim_code_event_ids(events, event_ids):
    """
    Description:
        return dictionay of event_ids that match the criteria of the eeglab file.
        Some files may not have all stim-code events, and to avoid errors when creating epochs, 
        this method will return a dictionary of compatible event_ids.
        
    Variables:
        events: ndarray of ints (n_events, 3)
        event_ids: dictionary of event_ids 
            ex. {'stim-code': 1}
    -----
    return: dictionary of event_ids
    """
    unique_events = np.unique(events[:,-1])
    event_id_values = event_ids.values()
    event_dict = {}
    
    for event_id in unique_events:
        if event_id in event_id_values:
            key = list(event_ids.keys())[event_id-1]
            event_dict[key] = event_id
            
    return event_dict

In [ ]:
def artifact_removal(eeglab):
    """
    NOTE: use this method after creating epoch objects to speed up process!
    
    Description:
        Perform notch filter, frequenc filter, and ICA 
        
    Variables: 
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    returns:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object

    """
    
    raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
    raw.filter(5,100, fir_design='firwin')
    raw.resample(256, n_jobs=10)
    ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
    ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
    ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                 ecg_score_func='pearsonr', skew_criterion=None, 
                 kurt_criterion=None, var_criterion=None,add_nodes=None)
    
    return ica.apply(raw)

In [ ]:
def create_epochs(eeglab, events, event_id):
    """
    Description: 
        create mne.epoch object from eeglab data.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
        events: array of int, shape (n_events, 3)
        event_id: dictionary of events 
            Keys are name of event as string, value is event_id as an int.
    
    returns:
        mne.epoch object
            
    """
    tmax = (1/raw.info['sfreq']) * 499 # tmax 1/sfrq * desired epoch length
    return mne.Epochs(raw, 
                      events=events,
                      event_id = new_stim_code_event_ids, 
                      preload = True, 
                      tmin=0, 
                      tmax=tmax) 
    

In [ ]:
# tmax = (1/raw.info['sfreq']) * 499 # tmax 1/sfrq * desired epoch length
# epoch = mne.Epochs(raw, events=new_events, event_id = new_stim_code_event_ids, preload = True, tmin=0, tmax=tmax) 

In [ ]:
def remove_artifacts_and_create_epochs(eeglab):
    """
    Description:
        Handles the start to finish process of finding events, event_ids, 
        artifact removal, and creaing epoch object.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    
    Returns:
        mne.epoch object
    """
    events = filter_bad_trials(eeglab)
    stim_code_events = events_with_stim_codes(events)
    stim_code_event_ids = get_stim_code_event_ids(stim_code_events,new_stim_code_event_ids)
    raw_ica = artifact_removal(eeglab= eeglab)
    
    return create_epochs(eeglab= raw_ica, events= stim_code_events, event_id= stim_code_event_ids)
    

In [ ]:
# eeglab_files = list_eeglab_files(path)
# eeglab_files

In [ ]:
# raw = mne.io.read_raw_eeglab(eeglab_files[3],preload = True)

In [ ]:
# epoch = remove_artifacts_and_create_epochs(eeglab=raw)

In [ ]:
# np.unique(epoch.events[:,-1])

In [ ]:
# 5565/9

In [ ]:
event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [ ]:
# get_stim_code_event_ids(new_events, new_stim_code_event_ids)

In [ ]:
# list(new_stim_code_event_ids.keys())

In [ ]:
# new_stim_code_event_ids.values()

In [ ]:
# new_events = events_with_stim_codes(filtered_events)
# np.unique(new_events[:,-1])

In [ ]:
# for set_file in eeglab_files[0]:
#     try:
#         start = time.time()
#         raw = mne.io.read_raw_eeglab(set_file, preload=True)
#         raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
#         raw.filter(5,100, fir_design='firwin')
#         raw.resample(256, n_jobs=10)
#         ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
#         ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
#         ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#                      ecg_score_func='pearsonr', skew_criterion=None, 
#                      kurt_criterion=None, var_criterion=None,add_nodes=None)
#         raw_ica = ica.apply(raw)

#         m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1, initial_event=True)
#         epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
# #         epoch.save(set_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#         end = time.time()
#         print("SUCESSFULLY LOADED time elapsed: {0}", (end - start))
#     except :
#         print("***Error converting file: {0}***".format(set_file))
        

In [ ]:
# start = time.time()
# # raw = mne.io.read_raw_eeglab(, preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# # m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
# epoch = mne.Epochs(raw, events=new_events, event_id =new_stim_code_event_ids , preload = True, tmin=0, tmax=1,)
# end = time.time()
# print("time elapsed: {0}", (end - start))

## check if saving and loading epoch file works.

In [ ]:
# os.chdir(epoch_destination)

In [ ]:

# epoch.save('20131216_1310_epo.fif',verbose = True)

In [ ]:
# epoch_files = os.listdir(epoch_destination)

In [ ]:
# epoch_file = epoch_files[0]
# epoch_ = mne.read_epochs(epoch_file,preload=True)

In [ ]:
# epoch_.plot(title="epochs");

In [ ]:
# for set_file in eeglab_files:
#     print(set_file.split('\\')[-1].split('.')[0] + "_epo.fif")

In [ ]:
# # start = time.time()
# raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1.5, initial_event=True)
# epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)

## Convert Raw EEGLAB Data

In [ ]:
# #get eeglab data
# path = 'C:\\Users\Emmanuil\Desktop\School\Fall 2018\Senior Project\EEGLAB'
# eeglab_files = list_eeglab_files(path)

In [ ]:
# # save new epoch files in the directory specified below. 
# os.chdir('D:\stim_code_epochs')
# for file in eeglab_files:
#     try:
#         raw = mne.io.read_raw_eeglab(file, preload = True)
#         epoch = remove_artifacts_and_create_epochs(raw)
#         epoch.save(file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#     except:
#         print("Could not convert {0}".format(file))